In [22]:
import folium
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors

In [23]:
# -----------------------------
# 1. CSV 불러오기
# -----------------------------
hangang_df = pd.read_csv("../../../bertopic_results/sentiment/hangang_ranking_topic.csv")
nonhangang_df = pd.read_csv("../../../bertopic_results/sentiment/nonhangang_area_ranking_topic.csv")

# -----------------------------
# 2. GeoDataFrame 불러오기
# -----------------------------
gu = gpd.read_file("../../../map/result/gu.gpkg", layer="gu").to_crs(4326)
parks = gpd.read_file("../../../map/result/parks.gpkg", layer="hangang_parks").to_crs(4326)

In [24]:
# -----------------------------
# 3. merge
# -----------------------------
# 자치구별 여가도_조정점수

gu = gu.merge(
    nonhangang_df[["District", "eco_score_std"]],
    left_on="SIGUNGU_NM",
    right_on="District",
    how="left"
).drop(columns=["District"])

# 공원명: park_name 영문 → 한글 변환
park_name_map = {
    "Banpo": "반포한강공원", "Gangseo": "강서한강공원", "Gwangnaru": "광나루한강공원",
    "Ichon": "이촌한강공원", "Jamsil": "잠실한강공원", "Jamwon": "잠원한강공원",
    "Mangwon": "망원한강공원", "Nanji": "난지한강공원", "Ttukseom": "뚝섬한강공원",
    "Yanghwa": "양화한강공원", "Yeouido": "여의도한강공원"
}
parks["park_name"] = parks["park_name"].map(park_name_map)


In [25]:
# 공원명 종합_여가도_점수 merge
parks = parks.merge(
    hangang_df[["Park_nm", "eco_score_std"]],
    left_on="park_name",
    right_on="Park_nm",
    how="left"
).drop(columns=["Park_nm"])

# -----------------------------
# 4. colormap (구 + 공원 전체 기준)
# -----------------------------
all_min = min(gu["eco_score_std"].min(), parks["eco_score_std"].min())
all_max = max(gu["eco_score_std"].max(), parks["eco_score_std"].max())

cmap = plt.cm.Greens
norm = colors.Normalize(vmin=all_min, vmax=all_max)

In [26]:
# -----------------------------
# 5. folium 지도 생성
# -----------------------------
m = folium.Map(location=[37.5665,126.9780], zoom_start=11, tiles="CartoDB positron")

# 자치구 레이어
folium.GeoJson(
    gu,
    style_function=lambda f: {
        "fillColor": "#eeeeee" if pd.isna(f["properties"]["eco_score_std"]) else 
                     colors.rgb2hex(cmap(norm(f["properties"]["eco_score_std"]))),
        "color": "#666",
        "weight": 1.2,
        "fillOpacity": 1
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["SIGUNGU_NM", "eco_score_std"],
        aliases=["자치구", "최종 생태경관성 점수"]
    )
).add_to(m)

# 공원명 레이어
folium.GeoJson(
    parks,
    name="공원명",
    style_function=lambda f: {
        "fillColor": "#eeeeee" if pd.isna(f["properties"]["eco_score_std"]) else 
                     colors.rgb2hex(cmap(norm(f["properties"]["eco_score_std"]))),
        "color": "#666",
        "weight": 1.2,
        "fillOpacity": 1
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["park_name", "eco_score_std"],
        aliases=["공원", "최종 생태경관성 점수"]
    )
).add_to(m)

In [27]:
# 저장
folium.LayerControl(collapsed=False).add_to(m)
m.save("../../result/NAT/nat_score_visualization.html")
print("nat_score_visualization.html 저장 완료")

nat_score_visualization.html 저장 완료
